In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import numpy as np
import datetime
import yfinance as yf
import hvplot.pandas

ModuleNotFoundError: No module named 'hvplot'

In [ ]:
# Load .env enviroment variables
load_dotenv()

In [ ]:
# defining the date for yahoo finance API
today = datetime.date.today().isoformat()
today

In [2]:
# downloading all stock, crypto and index data
data = yf.download("DX-Y.NYB COIN GC=F ^GSPC ^IXIC BTC-USD", start="2014-09-17", end=today)

NameError: name 'today' is not defined

In [5]:
# trimming the data
securities_data = data['Close']
securities_data = securities_data.rename(columns={'DX-Y.NYB': 'USDX','GC=F': 'GCF', '^GSPC': 'GSPC', '^IXIC': 'IXIC'}).copy(deep=True)
securities_data.tail()

,BTC-USD,COIN,USDX,GCF,GSPC,IXIC
Date,,,,,,
2022-04-10,42207.671875,NaN,NaN,NaN,NaN,NaN
2022-04-11,39521.902344,153.869995,99.930000,1944.300049,4412.529785,13411.959961
2022-04-12,40127.183594,149.850006,100.290001,1972.099976,4397.450195,13371.570312
2022-04-13,41166.730469,154.789993,99.879997,1981.000000,4446.589844,13643.589844
2022-04-14,39935.515625,NaN,100.320000,1972.900024,NaN,NaN


In [6]:
# deleting the weekends from the data
securities_data = securities_data.dropna(subset=['USDX', 'GCF', 'GSPC', 'IXIC'], how='all')
securities_data.head()

,BTC-USD,COIN,USDX,GCF,GSPC,IXIC
Date,,,,,,
2014-09-17,457.334015,NaN,84.699997,1234.400024,2001.569946,4562.189941
2014-09-18,424.440002,NaN,84.320000,1225.699951,2011.359985,4593.430176
2014-09-19,394.795990,NaN,84.800003,1215.300049,2010.400024,4579.790039
2014-09-22,402.152008,NaN,84.669998,1216.800049,1994.290039,4527.689941
2014-09-23,435.790985,NaN,84.660004,1221.000000,1982.770020,4508.689941


In [7]:
securities_pct = securities_data.pct_change()
securities_pct = securities_data.iloc[1:]
securities_pct.head()

,BTC-USD,COIN,USDX,GCF,GSPC,IXIC
Date,,,,,,
2014-09-18,424.440002,NaN,84.320000,1225.699951,2011.359985,4593.430176
2014-09-19,394.795990,NaN,84.800003,1215.300049,2010.400024,4579.790039
2014-09-22,402.152008,NaN,84.669998,1216.800049,1994.290039,4527.689941
2014-09-23,435.790985,NaN,84.660004,1221.000000,1982.770020,4508.689941
2014-09-24,423.204987,NaN,85.050003,1218.599976,1998.300049,4555.220215


In [8]:
# creating the inflation data frame
inflation = pd.read_csv("US_Inflation.csv",index_col="Year", parse_dates=True,infer_datetime_format=True)
inflation.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
Year,,,,,,,,,,,,,,
2012-01-01,227.842,228.329,228.807,229.187,228.713,228.524,228.590,229.918,231.015,231.638,231.249,231.221,NaN,NaN
2013-01-01,231.679,232.937,232.282,231.797,231.893,232.445,232.900,233.456,233.544,233.669,234.100,234.719,NaN,NaN
2014-01-01,235.288,235.547,236.028,236.468,236.918,237.231,237.498,237.460,237.477,237.430,236.983,236.252,NaN,NaN
2015-01-01,234.747,235.342,235.976,236.222,237.001,237.657,238.034,238.033,237.498,237.733,238.017,237.761,NaN,NaN
2016-01-01,237.652,237.336,238.080,238.992,239.557,240.222,240.101,240.545,241.176,241.741,242.026,242.637,NaN,NaN


In [9]:
# selecting the total inflation, renaming the column and slicing the data to match the other df
inflation_df = inflation.drop(inflation.iloc[:, 12:13], axis = 1)
inflation_df.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF2
Year,,,,,,,,,,,,,
2012-01-01,227.842,228.329,228.807,229.187,228.713,228.524,228.590,229.918,231.015,231.638,231.249,231.221,NaN
2013-01-01,231.679,232.937,232.282,231.797,231.893,232.445,232.900,233.456,233.544,233.669,234.100,234.719,NaN
2014-01-01,235.288,235.547,236.028,236.468,236.918,237.231,237.498,237.460,237.477,237.430,236.983,236.252,NaN
2015-01-01,234.747,235.342,235.976,236.222,237.001,237.657,238.034,238.033,237.498,237.733,238.017,237.761,NaN
2016-01-01,237.652,237.336,238.080,238.992,239.557,240.222,240.101,240.545,241.176,241.741,242.026,242.637,NaN


# Data Analysis


## Plot Daily Price Change Per Asset

In [83]:
#Plot daily closing values
securities_plot = securities_data.hvplot.line(figsize=(20,10), ylabel='Asset Price',xlabel='Time',title='Asset Performance Over Time',width=1000,height=500)
#inflation_plot no longer works with new data
    #inflation_plot = inflation_df.hvplot.line(xlabel='Date',figsize=(20,10), title='Inflation Over Time')

In [84]:
securities_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

### Chart beta and correlation of USD VS Bitcoin

In [95]:
#Chart moving correlation of S&P and BTC

# Drop unrelated columns
moving_correlation_USD = securities_pct.drop(["COIN",'GSPC','GCF','IXIC','BTC-USD'], axis=1)

#Find moving correlation
moving_correlation_USD = moving_correlation_USD.rolling(window=30).corr()

#Plot moving correlation
moving_correlation_USD.hvplot.line(title='Moving Correlation Of USD and Bitcoin',xlabel='Date',ylabel='Correlation',width=1000,height=300)

:Curve   [Date]   (USDX)

In [89]:
#Chart rolling beta of USD in relation to BTC

# Find rolling BTC variance
rolling_btc_variance = securities_pct['BTC-USD'].rolling(window=30).var()

# Calculate USD rolling covariance
rolling_usd_covariance = usd_df.rolling(window=30).cov(btc_df)

#Calculate USD rolling beta
rolling_usd_beta = rolling_usd_covariance/ rolling_btc_variance

#Plot rolling beta
rolling_usd_beta.hvplot.line(title='Moving Beta Of USD and Bitcoin Over Time',xlabel='Time',ylabel='Beta',width=1000,height=300)


:Curve   [Date]   (0)

### Chart of moving correlation and beta between S&P and NASDAQ and BTC

In [94]:
#Chart moving correlation of S&P and BTC

# Drop unrelated columns
moving_correlation_sp = securities_pct.drop(["COIN",'USDX','GCF','IXIC','BTC-USD'], axis=1)

#Find moving correlation
moving_correlation_sp = moving_correlation_sp.rolling(window=30).corr()

#Plot moving correlation
moving_correlation_sp.hvplot.line(title='Moving Correlation Of S&P 500 and Bitcoin',xlabel='Date',ylabel='Beta',width=1000,height=300)

:Curve   [Date]   (GSPC)

In [93]:
#Chart moving correlation of Nasdaq and BTC

# Drop unrelated columns
moving_correlation_nq = securities_pct.drop(["COIN",'USDX','GCF','GSPC','BTC-USD'], axis=1)

#Find moving correlation
moving_correlation_nq = moving_correlation_nq.rolling(window=30).corr()

#Plot moving correlation
moving_correlation_nq.hvplot.line(title='Moving Correlation Of Nasdaq and Bitcoin',xlabel='Date',ylabel='Beta',width=1000,height=300,color='blue')

:Curve   [Date]   (IXIC)

### Chart of All Asset Beta

In [23]:
btc_variance = securities_pct['BTC-USD'].var()

#create dfs for each column
btc_df = securities_pct['BTC-USD']
coin_df = securities_pct['COIN']
usd_df = securities_pct['USDX']
gold_df = securities_pct['GCF']
sp_500_df = securities_pct['GSPC']
nasdaq_df = securities_pct['IXIC']

#Calculate covariance of each asset in relation to bitcoin
coin_covariance = coin_df.cov(btc_df)
usd_covariance = usd_df.cov(btc_df)
gold_covariance = gold_df.cov(btc_df)
sp_500_covariance = sp_500_df.cov(btc_df)
nasdaq_covariance = nasdaq_df.cov(btc_df)


#Calculate Beta

coin_beta = coin_covariance/ btc_variance
usd_beta = usd_covariance/ btc_variance
gold_beta = gold_covariance/ btc_variance
sp_500_beta = sp_500_covariance/ btc_variance
nasdaq_beta = nasdaq_covariance/ btc_variance

# Collect data into a dataframe

data = {'Asset':['COIN','USD','GOLD','S&P 500','NASDAQ'],'Beta':[coin_beta,usd_beta,gold_beta,sp_500_beta,nasdaq_beta]}
beta_df = pd.DataFrame(data)
colors = ['#A6F7F5','#A6D0F7','#A6A8F7','#CCA6F7','#F5A6F7']
beta_df.hvplot.bar(x='Asset',y='Beta',color=colors, figsize=(20,10), title='Asset Betas In Relation to Bitcoin')

:Bars   [Asset]   (Beta)

### Chart Rolling Beta Between Nasdaq and S&P in relation to BTC

In [74]:
# Create a dataframe of just Nasdaq and S&P percent changes
sp_nq_pct_change = securities_pct.drop(['COIN','USDX','GCF','BTC-USD'],axis=1)

sp_nq_pct_change.head()

#Calculate the Coviariance of each column
nq_rolling_covariance = sp_nq_pct_change['IXIC'].rolling(window=30).cov(btc_df)
nq_rolling_covariance = nq_rolling_covariance.to_frame(name='NASDAQ')

sp_rolling_covariance = sp_nq_pct_change['GSPC'].rolling(window=30).cov(btc_df)
sp_rolling_covariance = sp_rolling_covariance.to_frame(name='S&P 500')


nq_sp_rolling_covariance = pd.concat([nq_rolling_covariance, sp_rolling_covariance], join='inner', axis=1)

nq_sp_rolling_covariance.hvplot.line(xlabel='Time',ylabel='Beta',color=colors, height=300,width=1000, title='Nasdaq and S&P 500 Beta in Relation to Bitcoin')

:NdOverlay   [Variable]
   :Curve   [Date]   (value)